# VAE for collaborative filtering

This implemetation is inspired by the paper of G. Krishnan et al. "Variational Autoencoders for Collaborative Filtering" [1] an their implemetation [2]. It works with implicit feedback data.

In [116]:
import os

import bottleneck as bn
import pandas as pd

import tensorflow as tf
from tensorflow.contrib.layers import apply_regularization, l2_regularizer

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [194]:
from keras.losses import binary_crossentropy
from keras import backend as K
from keras.layers import Lambda, Input, Dense, Dropout
from keras.models import Model
from keras.callbacks import LambdaCallback, EarlyStopping, Callback
from keras.utils import plot_model
from tensorflow.metrics import recall_at_k

In [58]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [59]:
DATA_PATH_SMALL='./data/ml-1m/'
DATA_PATH_LARGE='./data/ml-20m/'
DATA_FILE_NAME='ratings.dat'

In [60]:
data=pd.read_csv(DATA_PATH_SMALL + DATA_FILE_NAME, sep='::',names=['userId','movieId','rating','timestamp'])

/Users/install/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [61]:
data.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


We don't need explicit data (e.g. specific ratings), so we will convert the rating to a binary property. Every rating greater or equal to 4 becomes 1 and each rating smaller than 4 becomes 0.

In [62]:
bin_data=data[data['rating'] >= 4]
bin_data=bin_data.assign(rating=1)

In [63]:
bin_data.head()

,userId,movieId,rating,timestamp
0,1,1193,1,978300760
3,1,3408,1,978300275
4,1,2355,1,978824291
6,1,1287,1,978302039
7,1,2804,1,978300719


In [64]:
click_matrix=bin_data.pivot(index='movieId', columns='userId', values='rating')

In [65]:
click_matrix=click_matrix.fillna(0)

In [66]:
click_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Drop all rows and columns with less then 5 ratings. This is due to the cold start problem. Removing these entries will cause other rows/columns to fall under 5 ratings, but this is just a small portion.**

In [67]:
min_u=5
min_i=5

In [68]:
click_matrix.shape

(3533, 6038)

In [69]:
click_matrix=click_matrix.loc[(click_matrix.sum(axis=1) >= min_i), (click_matrix.sum(axis=0) >= min_u)]

In [70]:
click_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
(n_items,n_users)=click_matrix.shape
n_entries=click_matrix.values.sum()

sparsity=n_entries/(n_items*n_users)

print(f'There are {n_items} items and {n_users} users in the dataframe. The sparsity of the matrix is {sparsity: .4f}')

There are 3125 items and 6034 users in the dataframe. The sparsity of the matrix is  0.0305


**Shuffle the whole matrix**

In [72]:
click_matrix=click_matrix.reindex(np.random.RandomState(seed=42).permutation(click_matrix.index), axis='rows')
click_matrix=click_matrix.reindex(np.random.RandomState(seed=42).permutation(click_matrix.columns), axis='columns')
click_matrix.head()

userId,5323,3219,5123,1667,3661,5632,5143,2446,5257,1371,...,4430,5340,467,5740,3093,3774,5197,5232,5396,861
movieId,,,,,,,,,,,,,,,,,,,,,
1036,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Split train, validation and test sets

In [73]:
test_set_size=600
val_set_size=600
train_set_size=n_users-test_set_size-val_set_size

In [74]:
train_set=click_matrix.iloc[:,:train_set_size]
val_set=click_matrix.iloc[:,train_set_size:train_set_size+val_set_size]
test_set=click_matrix.iloc[:,train_set_size+val_set_size:]
print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

(3125, 4834)
(3125, 600)
(3125, 600)


## Mask a part of the test and validation set, to evaluate our model

In [85]:
with_held_rate=0.2

mask_ones=pd.DataFrame(1, index=val_set.index.values[:int(np.ceil(len(val_set.index)*with_held_rate))], columns=val_set.columns)
mask_zeros=pd.DataFrame(0, index=val_set.index.values[int(np.ceil(len(val_set.index)*with_held_rate)):], columns=val_set.columns)

mask=pd.concat([mask_ones,mask_zeros])
mask.shape

(3125, 600)

In [293]:
val_set_masked=val_set-mask
held_out_val=val_set_masked.replace(-1,0)

In [100]:
mask_ones=pd.DataFrame(1, index=test_set.index.values[:int(np.ceil(len(test_set.index)*with_held_rate))], columns=test_set.columns)
mask_zeros=pd.DataFrame(0, index=test_set.index.values[int(np.ceil(len(test_set.index)*with_held_rate)):], columns=test_set.columns)

mask=pd.concat([mask_ones,mask_zeros])
mask.shape

(3125, 600)

In [292]:
test_set_masked=test_set-mask
held_out_test=test_set_masked.replace(-1,0)


# Metric

We are using the **Recall@R** metric, which compares the top **R** recommended items with the held-out ratings to see how many of the recommended items are in the held-out set. Both implementations are from [2].

In [205]:
def Recall_at_k(X_pred, heldout_batch, k=20):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0)
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

In [281]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=20):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk] * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in range(len(np.count_nonzero(heldout_batch,axis=1)))])
    print(IDCG)
    return DCG / IDCG

In [24]:
# This code is from https://keras.io/examples/variational_autoencoder/
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

## Annealing

In [222]:
# the total number of gradient updates for annealing
total_anneal_steps = 5000
# largest annealing parameter
anneal_cap = 0.2

In [236]:
beta = K.variable(1., dtype=tf.float32)

def changeBeta(batch,logs):
    #maybe use epoch+1, because it starts with 0
    if total_anneal_steps > 0:
        anneal = min(anneal_cap, 1. * batch / total_anneal_steps)
    else:
        anneal = anneal_cap
    
    K.set_value(beta, anneal)
    
betaChanger = LambdaCallback(on_batch_end=changeBeta)

In [213]:
# Custom loss for VAE
def vae_loss(y_true, y_pred):
    rec_loss = binary_crossentropy(y_true, y_pred)
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    
    total_loss = K.mean(rec_loss + beta*kl_loss)
    
    return total_loss
    

## Model

In [237]:
# network parameters
input_shape = (n_items, )
dropout_rate = 0.5
intermediate_dim = 120
batch_size = 128
latent_dim = 40
epochs = 50

# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
drop = Dropout(dropout_rate, input_shape=input_shape)(inputs)
x = Dense(intermediate_dim, activation='relu')(drop)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(n_items, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 3125)         0                                            
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 3125)         0           encoder_input[0][0]              
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 120)          375120      dropout_5[0][0]                  
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 40)           4840        dense_16[0][0]                   
__________________________________________________________________________________________________
z_log_var 

### Training

In [295]:
# Quick hack to get custom metric for evaluation
class RecallMetric(Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(vae.predict(X_val))

        recall = Recall_at_k(y_predict,y_val)

        self._data.append({
            f'batch-{batch}': np.mean(recall),
        })
        return

    def get_data(self):
        return self._data

In [239]:
vae.compile(optimizer='adam', loss=vae_loss)
earlyStopping = EarlyStopping(monitor='val_loss', patience=5)
vae.summary()
plot_model(vae,
           to_file='vae_mlp.png',
           show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 3125)              0         
_________________________________________________________________
encoder (Model)              [(None, 40), (None, 40),  384800    
_________________________________________________________________
decoder (Model)              (None, 3125)              383045    
Total params: 767,845
Trainable params: 767,845
Non-trainable params: 0
_________________________________________________________________


In [240]:
recall=RecallMetric()

train_set_transposed = train_set.T
val_set_transposed = val_set.T
held_out_val_set_transposed = held_out_val.T

vae.fit(train_set_transposed,
                train_set_transposed,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(held_out_val_set_transposed, val_set_transposed),
                callbacks = [betaChanger, earlyStopping, recall])

vae.save_weights('vae_mlp_mnist.h5')

Train on 4834 samples, validate on 600 samples
Epoch 1/50
 256/4834 [>.............................] - ETA: 3:35 - loss: 1.8937

/Users/install/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.044744). Check your callbacks.
  % delta_t_median)


4834/4834 [==============================] - 21s 4ms/step - loss: 0.5087 - val_loss: 0.2382
Epoch 2/50
4834/4834 [==============================] - 8s 2ms/step - loss: 0.1693 - val_loss: 0.1419
Epoch 3/50
4834/4834 [==============================] - 9s 2ms/step - loss: 0.1301 - val_loss: 0.1247
Epoch 4/50
4834/4834 [==============================] - 9s 2ms/step - loss: 0.1219 - val_loss: 0.1180
Epoch 5/50
4834/4834 [==============================] - 9s 2ms/step - loss: 0.1183 - val_loss: 0.1152
Epoch 6/50
4834/4834 [==============================] - 9s 2ms/step - loss: 0.1162 - val_loss: 0.1136
Epoch 7/50
4834/4834 [==============================] - 10s 2ms/step - loss: 0.1149 - val_loss: 0.1117
Epoch 8/50
4834/4834 [==============================] - 9s 2ms/step - loss: 0.1140 - val_loss: 0.1108
Epoch 9/50
4834/4834 [==============================] - 9s 2ms/step - loss: 0.1137 - val_loss: 0.1104
Epoch 10/50
4834/4834 [==============================] - 9s 2ms/step - loss: 0.1132 - val_l

In [294]:
training_data=recall.get_data()
test_set_transposed = held_out_test.T
X_pred = vae.predict(test_set_transposed)

In [298]:
X_true=test_set.T.values
rec = Recall_at_k(X_pred, X_true, 20)

In [299]:
# ndcg=NDCG_binary_at_k_batch(X_pred, X_true, 20)

In [300]:
print(f'Recall@20 is {np.mean(rec)}')
print(f'Recall@50 is {np.mean(Recall_at_k(X_pred, X_true, 50))}')
print(f'Recall@100 is {np.mean(Recall_at_k(X_pred, X_true, 100))}')
# print(f'NDCG@20 is {np.mean(ndcg)}')


Recall@20 is 0.35779345150320385
Recall@50 is 0.34161196787135756
Recall@100 is 0.3757942694767363


## Sources

[1] Liang, D., Krishnan, R. G., Hoffman, M. D., & Jebara, T. (2018, April). Variational autoencoders for collaborative filtering. In Proceedings of the 2018 World Wide Web Conference (pp. 689-698). International World Wide Web Conferences Steering Committee.

[2] https://github.com/dawenl/vae_cf/

[3] https://keras.io/examples/variational_autoencoder/